In [94]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [174]:
uis = pd.read_csv('../data/UIS/uis.csv', index_col='Year_Country')
pisa = pd.read_csv('../data/pisa/pisa.csv', index_col='Year_Country')
world_bank = pd.read_csv('../data/world_bank/world_bank.csv', index_col='Year_Country')

In [175]:
df = pd.merge(pd.merge(uis, pisa, left_index=True, right_index=True),
              world_bank, left_index=True, right_index=True)

In [176]:
df.shape

(408, 57)

In [177]:
negative_features = ([feature for feature in df.columns if 'out' in feature][:-1]
                    + [feature for feature in df.columns if 'petition' in feature]
                    + [feature for feature in df.columns if 'peaters' in feature])

In [178]:
for feature in negative_features:
    df[feature] = 1 - df[feature]

In [179]:
mm_scaler = preprocessing.MinMaxScaler(feature_range=(0.9, 1.1))
mm_df = pd.DataFrame(mm_scaler.fit_transform(df),
                     index=df.index, columns=df.columns)
mm_df.head()

,"Gross intake ratio to Grade 1 of primary education, both sexes (%)","Gross enrolment ratio, primary, both sexes (%)","Gross enrolment ratio, primary to tertiary, both sexes (%)","Gross enrolment ratio, primary and secondary, both sexes (%)","Gross enrolment ratio, upper secondary, both sexes (%)","Gross enrolment ratio, lower secondary, both sexes (%)","Gross enrolment ratio, tertiary, both sexes (%)","Net enrolment rate, primary, both sexes (%)","Net enrolment rate, secondary, both sexes (%)","Net enrolment rate, lower secondary, both sexes (%)",...,Expenditure on secondary education (% of government expenditure on education),"Government expenditure per student, secondary (% of GDP per capita)",Expenditure on primary education (% of government expenditure on education),"Government expenditure per student, primary (% of GDP per capita)","Current education expenditure, total (% of total expenditure in public institutions)","Current education expenditure, tertiary (% of total expenditure in tertiary public institutions)","Current education expenditure, secondary (% of total expenditure in secondary public institutions)","Current education expenditure, primary (% of total expenditure in primary public institutions)","Literacy rate, adult total (% of people ages 15 and above)","Literacy rate, youth total (% of people ages 15-24)"
Year_Country,,,,,,,,,,,,,,,,,,,,,
2000_Albania,0.942660,0.965837,0.924936,0.925246,0.906309,0.952408,0.916788,1.054992,0.989347,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000_Algeria,0.931875,0.966554,NaN,0.931706,0.903440,0.938056,NaN,0.981459,NaN,NaN,...,NaN,0.954890,NaN,0.922813,NaN,NaN,NaN,NaN,NaN,NaN
2000_Argentina,0.968821,1.013195,0.990274,1.009598,0.937758,0.993139,0.982925,1.090874,1.029752,1.05025,...,0.973779,0.948253,1.004107,0.936290,NaN,NaN,NaN,NaN,NaN,NaN
2000_Australia,NaN,0.949720,NaN,NaN,NaN,NaN,NaN,1.035708,NaN,NaN,...,0.983932,NaN,0.996720,0.960785,1.059723,1.064629,1.066200,1.068148,NaN,NaN
2000_Austria,NaN,0.961927,NaN,0.985765,0.962058,0.963278,NaN,NaN,1.061749,NaN,...,1.007935,0.994732,0.931337,0.992694,1.071869,1.088068,1.070098,1.068489,NaN,NaN


In [180]:
school_climate = list(uis.columns[:-2])
# school_climate

In [181]:
school_resources = list(world_bank.columns[:-2])
# school_resources

In [182]:
learning_outcomes = list(pisa.columns) + list(world_bank.columns[-2:])
# learning_outcomes

In [183]:
mm_df['climate_index'] = 1

for feature in school_climate:
    mm_df['climate_index'] = mm_df['climate_index'].multiply(mm_df[feature], fill_value=1)

In [184]:
mm_df['resources_index'] = 1

for feature in school_resources:
    mm_df['resources_index'] = mm_df['resources_index'].multiply(mm_df[feature], fill_value=1)

In [185]:
mm_df['learning_index'] = 1

for feature in learning_outcomes:
    mm_df['learning_index'] = mm_df['learning_index'].multiply(mm_df[feature], fill_value=1)

In [186]:
mm_df

,"Gross intake ratio to Grade 1 of primary education, both sexes (%)","Gross enrolment ratio, primary, both sexes (%)","Gross enrolment ratio, primary to tertiary, both sexes (%)","Gross enrolment ratio, primary and secondary, both sexes (%)","Gross enrolment ratio, upper secondary, both sexes (%)","Gross enrolment ratio, lower secondary, both sexes (%)","Gross enrolment ratio, tertiary, both sexes (%)","Net enrolment rate, primary, both sexes (%)","Net enrolment rate, secondary, both sexes (%)","Net enrolment rate, lower secondary, both sexes (%)",...,"Government expenditure per student, primary (% of GDP per capita)","Current education expenditure, total (% of total expenditure in public institutions)","Current education expenditure, tertiary (% of total expenditure in tertiary public institutions)","Current education expenditure, secondary (% of total expenditure in secondary public institutions)","Current education expenditure, primary (% of total expenditure in primary public institutions)","Literacy rate, adult total (% of people ages 15 and above)","Literacy rate, youth total (% of people ages 15-24)",climate_index,resources_index,learning_index
Year_Country,,,,,,,,,,,,,,,,,,,,,
2000_Albania,0.942660,0.965837,0.924936,0.925246,0.906309,0.952408,0.916788,1.054992,0.989347,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.817519,0.883472,0.930328
2000_Algeria,0.931875,0.966554,NaN,0.931706,0.903440,0.938056,NaN,0.981459,NaN,NaN,...,0.922813,NaN,NaN,NaN,NaN,NaN,NaN,0.533988,0.881185,1.000000
2000_Argentina,0.968821,1.013195,0.990274,1.009598,0.937758,0.993139,0.982925,1.090874,1.029752,1.050250,...,0.936290,NaN,NaN,NaN,NaN,NaN,NaN,3.008766,0.771763,1.000000
2000_Australia,NaN,0.949720,NaN,NaN,NaN,NaN,NaN,1.035708,NaN,NaN,...,0.960785,1.059723,1.064629,1.066200,1.068148,NaN,NaN,1.014587,1.176635,1.077049
2000_Austria,NaN,0.961927,NaN,0.985765,0.962058,0.963278,NaN,NaN,1.061749,NaN,...,0.992694,1.071869,1.088068,1.070098,1.068489,NaN,NaN,1.454729,1.207459,1.047541
2000_Belgium,NaN,0.966278,1.031414,1.070535,1.011220,1.051420,0.991713,1.088447,1.078004,1.061685,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.232634,1.000000,1.059836
2000_Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.926338,1.064717,1.088664,1.067173,1.052503,1.001180,0.959258,0.874982,1.094753,0.930475
2000_Bulgaria,0.946939,0.961114,0.957322,0.971959,0.968890,0.935294,0.966997,1.048564,1.055083,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.798407,1.000000,0.996721
2000_Canada,0.943991,0.949125,0.993906,0.987487,0.972397,0.955883,0.992876,1.099682,NaN,NaN,...,NaN,1.079309,NaN,NaN,NaN,NaN,NaN,1.281197,1.036715,1.081967


In [187]:
mm_df['ed_qual_index'] = (mm_df['climate_index'] + mm_df['resources_index']
                          + mm_df['learning_index']) / 3

In [192]:
list(mm_df.sort_values('ed_qual_index', ascending=False).index[:20])

['2015_Finland',
 '2015_Denmark',
 '2015_Ireland',
 '2015_Belgium',
 '2015_Lithuania',
 '2015_Slovenia',
 '2006_Belgium',
 '2012_Belgium',
 '2015_Spain',
 '2015_Sweden',
 '2015_Australia',
 '2009_Belgium',
 '2012_Denmark',
 '2015_Italy',
 '2012_Finland',
 '2015_Norway',
 '2012_Kazakhstan',
 '2012_Lithuania',
 '2003_Belgium',
 '2009_Finland']

In [197]:
mm_df.corr().sort_values('ed_qual_index', ascending=False)['ed_qual_index'][:20]

ed_qual_index                                                                                                1.000000
climate_index                                                                                                0.995798
Rate of out-of-school children, adolescents and youth of primary and secondary school age, both sexes (%)    0.774698
Gross enrolment ratio, primary to tertiary, both sexes (%)                                                   0.751994
Gross enrolment ratio, upper secondary, both sexes (%)                                                       0.709892
Net enrolment rate, secondary, both sexes (%)                                                                0.700447
Rate of out-of-school adolescents and youth of secondary school age, both sexes (%)                          0.698867
Rate of out-of-school youth of upper secondary school age, both sexes (%)                                    0.683309
Total net enrolment rate, upper secondary, both sexes (%

In [207]:
years = [year for year in range(2000, 2016, 3)]
years

[2000, 2003, 2006, 2009, 2012, 2015]

In [212]:
for year in years:
    indices = [index for index in df.index if str(year) in index]
    mean = mm_df.loc[indices]['ed_qual_index'].mean()
    print(f"{year} mean: {mean}")

2000 mean: 1.2416421861443059
2003 mean: 1.3122056961233546
2006 mean: 1.3730055906331387
2009 mean: 1.4508077040539689
2012 mean: 1.5262739749161363
2015 mean: 1.8495199196126662
